# 1. Configuration

In [ ]:
import os
import gc
import cv2
import copy
import random
import numpy as np
import pandas as pd
import polars as pl
from pathlib import Path
import typing as tp
from io import BytesIO
from PIL import Image
import h5py
import timm
from time import time
from tqdm.notebook import tqdm
import shutil
import pickle

import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.model_selection import PredefinedSplit
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import VotingClassifier
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

import lightgbm as lgb
import catboost as cb
import xgboost as xgb

import optuna
import wandb
import torch
import torchvision
from torch import nn
from torch import optim
from torch.optim import lr_scheduler
from torch import amp
from torch.utils.data import Dataset
import torch.nn.functional as F
import albumentations as A
from albumentations.pytorch import ToTensorV2

pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

In [ ]:
class CFG:
    
    exp = 46
    
    is_kaggle = False
    is_infer = True
    enable_wandb = False
    if is_kaggle:
        OUTPUT_DIR = Path('/kaggle/input/isic2024-baseline')
        TRAIN_DIR = Path('/kaggle/input/isic-2024-challenge/train-image/image')
        TRAIN_HDF5 = Path('/kaggle/input/isic-2024-challenge/train-image.hdf5')
        TEST_HDF5 = Path('/kaggle/input/isic-2024-challenge/test-image.hdf5')
        TRAIN_META = Path('/kaggle/input/isic-2024-challenge/train-metadata.csv')
        TEST_META = Path('/kaggle/input/isic-2024-challenge/test-metadata.csv')
        SAMPLE_SUB = Path('/kaggle/input/isic-2024-challenge/sample_submission.csv')
        PRETRAINED_MODEL = ''
        
    else:
        OUTPUT_DIR = Path(f'/root/Development/Kaggle/ISIC2024/main/models/experiments/exp{exp}/outputs')
        OUTPUT_LOG = Path(f'/root/Development/Kaggle/ISIC2024/main/models/experiments/exp{exp}/log.txt')
        TRAIN_DIR = Path('/root/Development/Kaggle/ISIC2024/data/raw/train-image/image')
        TRAIN_HDF5 = Path('/root/Development/Kaggle/ISIC2024/data/raw/train-image.hdf5')
        TEST_HDF5 = Path('/root/Development/Kaggle/ISIC2024/data/raw/test-image.hdf5')
        TRAIN_META = Path('/root/Development/Kaggle/ISIC2024/data/raw/train-metadata.csv')
        TEST_META = Path('/root/Development/Kaggle/ISIC2024/data/raw/test-metadata.csv')
        SAMPLE_SUB = Path('/root/Development/Kaggle/ISIC2024/data/raw/sample_submission.csv')
        TRAIN_META_ADD = Path('/root/Development/Kaggle/ISIC2024/data/external/All/metadata.csv')
        TRAIN_HDF5_ADD = Path('/root/Development/Kaggle/ISIC2024/data/external/All/image.hdf5')
        TRAIN_HDF5_COMBINED = Path('/root/Development/Kaggle/ISIC2024/data/processed/train-image-combined.hdf5')
        # PRETRAINED_MODEL = ''
        PRETRAINED_MODEL = Path('/root/Development/Kaggle/ISIC2024/main/models/experiments/exp32/outputs/averaged_model.pth')
    
    es_patience = 5
    batch_size = 128 # 256
    max_epoch = 9
    n_folds = 5
    n_classes = 2
    random_seed = 42
    lr = 1.0e-04 # lr = 1.0e-03
    weight_decay = 1.0e-02 # default
    deterministic = True
    enable_amp = True
    view = True
    change_dataset = True
    standardization = True
    oversampling = False
    oversampling_percent = 3
    only_lesion_id = False
    alldata_isic_archive = False
    test_of_averaged_model = False
    train_with_fewdata: int|bool = False
    # train_with_fewdata: int|bool = 100000 # or False
    pretrained = True
    TTA: bool = True
    TTA_rate = {'None':0.7, 'with_train_aug':0.3}
    AUX_LOSS = False
    use_tabler = True
    image_pred_as_tabler_feature = False
    ensemble_image_table = True
    
    model_name = 'maxvit_rmlp_pico_rw_256.sw_in1k'
    output_dim_models = { 
        "resnet18.a1_in1k": 512,
        "efficientnet_b0.ra_in1k": 320,
        "tf_efficientnet_b0.ns_jft_in1k": 1280,
        # 'tf_efficientnet_b5.ns_jft_in1k': 1280,
        'tf_efficientnet_b3.ns_jft_in1k': 1536,
        'eva02_large_patch14_224.mim_in22k': 384,
        'eva02_large_patch14_448.mim_m38m_ft_in22k_in1k': 384,
        'maxvit_tiny_tf_224.in1k': 256,
        'maxvit_rmlp_nano_rw_256.sw_in1k': 256,
        'maxvit_rmlp_pico_rw_256.sw_in1k': 256,
        'maxvit_small_tf_224.in1k': 256,
    }
    
    # img_size = 224
    img_size = 256
    # img_size = 384
    interpolation = cv2.INTER_LINEAR
    
    if torch.cuda.is_available():
        device = "cuda"
    else:
        device = "cpu"
        
    
    wandb_config = {
    "es_patience" : es_patience,
    "batch_size" : batch_size, # 128 # 256
    "max_epoch" : max_epoch,
    "n_folds" : n_folds,
    "n_classes" : n_classes,
    "random_seed" : random_seed,
    "lr" : lr, # lr = 1.0e-04
    "weight_decay" : weight_decay, # default
    "deterministic" : deterministic,
    "enable_amp" : enable_amp,
    "view" : view,
    "change_dataset" : change_dataset,
    "standardization" : standardization,
    "oversampling" : oversampling,
    "oversampling_percent" : oversampling_percent,
    "only_lesion_id" : only_lesion_id,
    "alldata_isic_archive" : alldata_isic_archive,
    "test_of_averaged_model" : test_of_averaged_model,
    "train_with_fewdata" : train_with_fewdata, # 100000 or False
    "pretrained" : pretrained,
    "TTA" : TTA,
    "TTA_rate" : TTA_rate,
    "model_name" : model_name,
    "output_dim_models" : output_dim_models,
    "img_size" : img_size, # 224 # 256 # 384
    "interpolation" : interpolation,
    "AUX_LOSS" : AUX_LOSS,
    "use_tabler": use_tabler,
    "image_pred_as_tabler_feature": image_pred_as_tabler_feature,
    "ensemble_image_table": ensemble_image_table,
    }
    if enable_wandb:
        wandb.login()
        run = wandb.init(project="ISIC2024", name=f"experiment_{exp}", config=wandb_config)

In [ ]:

train_meta = pd.read_csv(CFG.TRAIN_META)
test_meta = pd.read_csv(CFG.TEST_META)
print(len(train_meta))
print(len(test_meta))

# head
display(train_meta[train_meta['target'] > 0.5].head(10))
display(train_meta.head())
display(test_meta.head())


if not CFG.is_infer:
    train_meta_add = pd.read_csv(CFG.TRAIN_META_ADD)
    print(len(train_meta_add))
    display(train_meta_add.head())

# 2. Preprocessing

### process additional data

In [ ]:
if not CFG.is_infer:

    # only benign or maligant
    train_meta_add = train_meta_add[train_meta_add['benign_malignant'].isin(['benign', 'malignant'])]
    # make the target column
    train_meta_add['target'] = train_meta_add['benign_malignant'].apply(lambda x: 1 if x == 'malignant' else 0)
    # assign difference patientid
    train_meta_add['patient_id'] = [
        f"example_{i+1}" if pd.isna(id) else id
        for i, id in enumerate(train_meta_add['patient_id'])
    ]

    train_meta_add['additional'] = 1

    display(train_meta_add.head())


### process original data

In [ ]:
if not CFG.is_infer:

    # delete clothes and others
    # List of IDs to drop
    ids_to_drop = ['ISIC_0573025', 'ISIC_1443812', 'ISIC_5374420', 'ISIC_2611119', 'ISIC_2691718', 'ISIC_9689783', 'ISIC_9520696', 'ISIC_8651165', 'ISIC_9385142', 'ISIC_9680590']
    train_meta = train_meta.drop(train_meta[train_meta['isic_id'].isin(ids_to_drop)].index)
    train_meta = train_meta.reset_index(drop=True)

    if CFG.train_with_fewdata:
        filtered_train_meta = train_meta[train_meta['lesion_id'].notnull()]
        print("len filtered_train_meta: ", len(filtered_train_meta))

        # Select 100 rows from the remaining DataFrame
        remaining_rows = train_meta[~train_meta.index.isin(filtered_train_meta.index)]
        selected_rows = remaining_rows.sample(n=CFG.train_with_fewdata - len(filtered_train_meta), random_state=CFG.random_seed).reset_index(drop=True)

        # Combine both DataFrames
        combined_df = pd.concat([filtered_train_meta, selected_rows]).reset_index(drop=True)
        train_meta = combined_df

    # only has lesion_id(strgong label)
    if CFG.only_lesion_id:
        train_meta = train_meta[train_meta['lesion_id'].notnull()].reset_index(drop=True)
        
        
        
    # add "additional" column
    train_meta['additional'] = 0

    print(len(train_meta))
    print(len(train_meta[train_meta['target']==1]))
    print(len(train_meta[train_meta['target']==1]) / len(train_meta) * 100, '%')
    display(train_meta.head())

### concat both data

In [ ]:
if not CFG.is_infer:

    concatenated = pd.concat([train_meta, train_meta_add], axis=0)
    if CFG.alldata_isic_archive:
        train_meta = concatenated.reset_index(drop=True)
        display(train_meta.head())
        display(train_meta.tail())

# 3. Split to fold

In [ ]:
if not CFG.is_infer:


    def split_fold(df:pd.DataFrame):
        # if 'fold' in df.columns:
        #     return df
        
        df['fold'] = -1
        # object
        skf = StratifiedGroupKFold(n_splits=CFG.n_folds, shuffle=True, random_state=CFG.random_seed)

        for i, (train_index, test_index) in enumerate(skf.split(df, df['target'], df['patient_id'])):
            df.loc[test_index, 'fold'] = i
        
        return df
            
    train_meta = split_fold(train_meta)
    display(train_meta.head().T)

    # check
    if CFG.view:
        print(train_meta.groupby('fold')['target'].value_counts().head(300))

### OverSampling

In [ ]:
if not CFG.is_infer:
    if CFG.oversampling:

        percent = CFG.oversampling_percent

        train_meta_mali = train_meta[train_meta['target']==1]
        for _ in range(100):
            train_meta = pd.concat([train_meta, train_meta_mali], axis=0)
            if len(train_meta[train_meta['target']==1]) / len(train_meta) * 100 >= percent:
                print(f'over {percent}% malignant')
                break

        print(len(train_meta))
        print(len(train_meta[train_meta['target']==1]))
        print(len(train_meta[train_meta['target']==1]) / len(train_meta) * 100, '%')

        del percent

        if CFG.view:
            print(train_meta.groupby('fold')['target'].value_counts().head(300))
            display(train_meta.head())

# 4 Tabler Model

### 4.1 Preparing

In [ ]:
id_col = 'isic_id'
target_col = 'target'
group_col = 'patient_id'

err = 1e-5
sampling_ratio = 0.01
seed = 42

num_cols = [
    'age_approx',                        # Approximate age of patient at time of imaging.
    'clin_size_long_diam_mm',            # Maximum diameter of the lesion (mm).+
    'tbp_lv_A',                          # A inside  lesion.+
    'tbp_lv_Aext',                       # A outside lesion.+
    'tbp_lv_B',                          # B inside  lesion.+
    'tbp_lv_Bext',                       # B outside lesion.+ 
    'tbp_lv_C',                          # Chroma inside  lesion.+
    'tbp_lv_Cext',                       # Chroma outside lesion.+
    'tbp_lv_H',                          # Hue inside the lesion; calculated as the angle of A* and B* in LAB* color space. Typical values range from 25 (red) to 75 (brown).+
    'tbp_lv_Hext',                       # Hue outside lesion.+
    'tbp_lv_L',                          # L inside lesion.+
    'tbp_lv_Lext',                       # L outside lesion.+
    'tbp_lv_areaMM2',                    # Area of lesion (mm^2).+
    'tbp_lv_area_perim_ratio',           # Border jaggedness, the ratio between lesions perimeter and area. Circular lesions will have low values; irregular shaped lesions will have higher values. Values range 0-10.+
    'tbp_lv_color_std_mean',             # Color irregularity, calculated as the variance of colors within the lesion's boundary.
    'tbp_lv_deltaA',                     # Average A contrast (inside vs. outside lesion).+
    'tbp_lv_deltaB',                     # Average B contrast (inside vs. outside lesion).+
    'tbp_lv_deltaL',                     # Average L contrast (inside vs. outside lesion).+
    'tbp_lv_deltaLB',                    #
    'tbp_lv_deltaLBnorm',                # Contrast between the lesion and its immediate surrounding skin. Low contrast lesions tend to be faintly visible such as freckles; high contrast lesions tend to be those with darker pigment. Calculated as the average delta LB of the lesion relative to its immediate background in LAB* color space. Typical values range from 5.5 to 25.+
    'tbp_lv_eccentricity',               # Eccentricity.+
    'tbp_lv_minorAxisMM',                # Smallest lesion diameter (mm).+
    'tbp_lv_nevi_confidence',            # Nevus confidence score (0-100 scale) is a convolutional neural network classifier estimated probability that the lesion is a nevus. The neural network was trained on approximately 57,000 lesions that were classified and labeled by a dermatologist.+,++
    'tbp_lv_norm_border',                # Border irregularity (0-10 scale); the normalized average of border jaggedness and asymmetry.+
    'tbp_lv_norm_color',                 # Color variation (0-10 scale); the normalized average of color asymmetry and color irregularity.+
    'tbp_lv_perimeterMM',                # Perimeter of lesion (mm).+
    'tbp_lv_radial_color_std_max',       # Color asymmetry, a measure of asymmetry of the spatial distribution of color within the lesion. This score is calculated by looking at the average standard deviation in LAB* color space within concentric rings originating from the lesion center. Values range 0-10.+
    'tbp_lv_stdL',                       # Standard deviation of L inside  lesion.+
    'tbp_lv_stdLExt',                    # Standard deviation of L outside lesion.+
    'tbp_lv_symm_2axis',                 # Border asymmetry; a measure of asymmetry of the lesion's contour about an axis perpendicular to the lesion's most symmetric axis. Lesions with two axes of symmetry will therefore have low scores (more symmetric), while lesions with only one or zero axes of symmetry will have higher scores (less symmetric). This score is calculated by comparing opposite halves of the lesion contour over many degrees of rotation. The angle where the halves are most similar identifies the principal axis of symmetry, while the second axis of symmetry is perpendicular to the principal axis. Border asymmetry is reported as the asymmetry value about this second axis. Values range 0-10.+
    'tbp_lv_symm_2axis_angle',           # Lesion border asymmetry angle.+
    'tbp_lv_x',                          # X-coordinate of the lesion on 3D TBP.+
    'tbp_lv_y',                          # Y-coordinate of the lesion on 3D TBP.+
    'tbp_lv_z',                          # Z-coordinate of the lesion on 3D TBP.+
]

new_num_cols = [
    'lesion_size_ratio',             # tbp_lv_minorAxisMM      / clin_size_long_diam_mm
    'lesion_shape_index',            # tbp_lv_areaMM2          / tbp_lv_perimeterMM **2
    'hue_contrast',                  # tbp_lv_H                - tbp_lv_Hext              abs
    'luminance_contrast',            # tbp_lv_L                - tbp_lv_Lext              abs
    'lesion_color_difference',       # tbp_lv_deltaA **2       + tbp_lv_deltaB **2 + tbp_lv_deltaL **2  sqrt  
    'border_complexity',             # tbp_lv_norm_border      + tbp_lv_symm_2axis
    'color_uniformity',              # tbp_lv_color_std_mean   / tbp_lv_radial_color_std_max

    'position_distance_3d',          # tbp_lv_x **2 + tbp_lv_y **2 + tbp_lv_z **2  sqrt
    'perimeter_to_area_ratio',       # tbp_lv_perimeterMM      / tbp_lv_areaMM2
    'area_to_perimeter_ratio',       # tbp_lv_areaMM2          / tbp_lv_perimeterMM
    'lesion_visibility_score',       # tbp_lv_deltaLBnorm      + tbp_lv_norm_color
    'symmetry_border_consistency',   # tbp_lv_symm_2axis       * tbp_lv_norm_border
    'consistency_symmetry_border',   # tbp_lv_symm_2axis       * tbp_lv_norm_border / (tbp_lv_symm_2axis + tbp_lv_norm_border)

    'color_consistency',             # tbp_lv_stdL             / tbp_lv_Lext
    'consistency_color',             # tbp_lv_stdL*tbp_lv_Lext / tbp_lv_stdL + tbp_lv_Lext
    'size_age_interaction',          # clin_size_long_diam_mm  * age_approx
    'hue_color_std_interaction',     # tbp_lv_H                * tbp_lv_color_std_mean
    'lesion_severity_index',         # tbp_lv_norm_border      + tbp_lv_norm_color + tbp_lv_eccentricity / 3
    'shape_complexity_index',        # border_complexity       + lesion_shape_index
    'color_contrast_index',          # tbp_lv_deltaA + tbp_lv_deltaB + tbp_lv_deltaL + tbp_lv_deltaLBnorm

    'log_lesion_area',               # tbp_lv_areaMM2          + 1  np.log
    'normalized_lesion_size',        # clin_size_long_diam_mm  / age_approx
    'mean_hue_difference',           # tbp_lv_H                + tbp_lv_Hext    / 2
    'std_dev_contrast',              # tbp_lv_deltaA **2 + tbp_lv_deltaB **2 + tbp_lv_deltaL **2   / 3  np.sqrt
    'color_shape_composite_index',   # tbp_lv_color_std_mean   + bp_lv_area_perim_ratio + tbp_lv_symm_2axis   / 3
    'lesion_orientation_3d',         # tbp_lv_y                , tbp_lv_x  np.arctan2
    'overall_color_difference',      # tbp_lv_deltaA           + tbp_lv_deltaB + tbp_lv_deltaL   / 3

    'symmetry_perimeter_interaction',# tbp_lv_symm_2axis       * tbp_lv_perimeterMM
    'comprehensive_lesion_index',    # tbp_lv_area_perim_ratio + tbp_lv_eccentricity + bp_lv_norm_color + tbp_lv_symm_2axis   / 4
    'color_variance_ratio',          # tbp_lv_color_std_mean   / tbp_lv_stdLExt
    'border_color_interaction',      # tbp_lv_norm_border      * tbp_lv_norm_color
    'border_color_interaction_2',
    'size_color_contrast_ratio',     # clin_size_long_diam_mm  / tbp_lv_deltaLBnorm
    'age_normalized_nevi_confidence',# tbp_lv_nevi_confidence  / age_approx
    'age_normalized_nevi_confidence_2',
    'color_asymmetry_index',         # tbp_lv_symm_2axis       * tbp_lv_radial_color_std_max

    'volume_approximation_3d',       # tbp_lv_areaMM2          * sqrt(tbp_lv_x**2 + tbp_lv_y**2 + tbp_lv_z**2)
    'color_range',                   # abs(tbp_lv_L - tbp_lv_Lext) + abs(tbp_lv_A - tbp_lv_Aext) + abs(tbp_lv_B - tbp_lv_Bext)
    'shape_color_consistency',       # tbp_lv_eccentricity     * tbp_lv_color_std_mean
    'border_length_ratio',           # tbp_lv_perimeterMM      / pi * sqrt(tbp_lv_areaMM2 / pi)
    'age_size_symmetry_index',       # age_approx              * clin_size_long_diam_mm * tbp_lv_symm_2axis
    'index_age_size_symmetry',       # age_approx              * tbp_lv_areaMM2 * tbp_lv_symm_2axis
]

cat_cols = ['sex', 'anatom_site_general', 'tbp_tile_type', 'tbp_lv_location', 'tbp_lv_location_simple', 'attribution']
norm_cols = [f'{col}_patient_norm' for col in num_cols + new_num_cols]
special_cols = ['count_per_patient']
feature_cols = num_cols + new_num_cols + cat_cols + norm_cols + special_cols

### 4.2 Read CSV and Preprocessing

In [ ]:
# recieve pd.df and creating feature columns in pl
def read_data(pd_df:pd.DataFrame):
    return (
        pl.from_pandas(pd_df)
        .with_columns(
            pl.col('age_approx').cast(pl.String).replace('NA', np.nan).cast(pl.Float64),
        )
        .with_columns(
            pl.col(pl.Float64).fill_nan(pl.col(pl.Float64).median()), # You may want to impute test data with train
        )
        .with_columns(
            lesion_size_ratio              = pl.col('tbp_lv_minorAxisMM') / pl.col('clin_size_long_diam_mm'),
            lesion_shape_index             = pl.col('tbp_lv_areaMM2') / (pl.col('tbp_lv_perimeterMM') ** 2),
            hue_contrast                   = (pl.col('tbp_lv_H') - pl.col('tbp_lv_Hext')).abs(),
            luminance_contrast             = (pl.col('tbp_lv_L') - pl.col('tbp_lv_Lext')).abs(),
            lesion_color_difference        = (pl.col('tbp_lv_deltaA') ** 2 + pl.col('tbp_lv_deltaB') ** 2 + pl.col('tbp_lv_deltaL') ** 2).sqrt(),
            border_complexity              = pl.col('tbp_lv_norm_border') + pl.col('tbp_lv_symm_2axis'),
            color_uniformity               = pl.col('tbp_lv_color_std_mean') / (pl.col('tbp_lv_radial_color_std_max') + err),
        )
        .with_columns(
            position_distance_3d           = (pl.col('tbp_lv_x') ** 2 + pl.col('tbp_lv_y') ** 2 + pl.col('tbp_lv_z') ** 2).sqrt(),
            perimeter_to_area_ratio        = pl.col('tbp_lv_perimeterMM') / pl.col('tbp_lv_areaMM2'),
            area_to_perimeter_ratio        = pl.col('tbp_lv_areaMM2') / pl.col('tbp_lv_perimeterMM'),
            lesion_visibility_score        = pl.col('tbp_lv_deltaLBnorm') + pl.col('tbp_lv_norm_color'),
            combined_anatomical_site       = pl.col('anatom_site_general') + '_' + pl.col('tbp_lv_location'),
            symmetry_border_consistency    = pl.col('tbp_lv_symm_2axis') * pl.col('tbp_lv_norm_border'),
            consistency_symmetry_border    = pl.col('tbp_lv_symm_2axis') * pl.col('tbp_lv_norm_border') / (pl.col('tbp_lv_symm_2axis') + pl.col('tbp_lv_norm_border')),
        )
        .with_columns(
            color_consistency              = pl.col('tbp_lv_stdL') / pl.col('tbp_lv_Lext'),
            consistency_color              = pl.col('tbp_lv_stdL') * pl.col('tbp_lv_Lext') / (pl.col('tbp_lv_stdL') + pl.col('tbp_lv_Lext')),
            size_age_interaction           = pl.col('clin_size_long_diam_mm') * pl.col('age_approx'),
            hue_color_std_interaction      = pl.col('tbp_lv_H') * pl.col('tbp_lv_color_std_mean'),
            lesion_severity_index          = (pl.col('tbp_lv_norm_border') + pl.col('tbp_lv_norm_color') + pl.col('tbp_lv_eccentricity')) / 3,
            shape_complexity_index         = pl.col('border_complexity') + pl.col('lesion_shape_index'),
            color_contrast_index           = pl.col('tbp_lv_deltaA') + pl.col('tbp_lv_deltaB') + pl.col('tbp_lv_deltaL') + pl.col('tbp_lv_deltaLBnorm'),
        )
        .with_columns(
            log_lesion_area                = (pl.col('tbp_lv_areaMM2') + 1).log(),
            normalized_lesion_size         = pl.col('clin_size_long_diam_mm') / pl.col('age_approx'),
            mean_hue_difference            = (pl.col('tbp_lv_H') + pl.col('tbp_lv_Hext')) / 2,
            std_dev_contrast               = ((pl.col('tbp_lv_deltaA') ** 2 + pl.col('tbp_lv_deltaB') ** 2 + pl.col('tbp_lv_deltaL') ** 2) / 3).sqrt(),
            color_shape_composite_index    = (pl.col('tbp_lv_color_std_mean') + pl.col('tbp_lv_area_perim_ratio') + pl.col('tbp_lv_symm_2axis')) / 3,
            lesion_orientation_3d          = pl.arctan2(pl.col('tbp_lv_y'), pl.col('tbp_lv_x')),
            overall_color_difference       = (pl.col('tbp_lv_deltaA') + pl.col('tbp_lv_deltaB') + pl.col('tbp_lv_deltaL')) / 3,
        )
        .with_columns(
            symmetry_perimeter_interaction = pl.col('tbp_lv_symm_2axis') * pl.col('tbp_lv_perimeterMM'),
            comprehensive_lesion_index     = (pl.col('tbp_lv_area_perim_ratio') + pl.col('tbp_lv_eccentricity') + pl.col('tbp_lv_norm_color') + pl.col('tbp_lv_symm_2axis')) / 4,
            color_variance_ratio           = pl.col('tbp_lv_color_std_mean') / pl.col('tbp_lv_stdLExt'),
            border_color_interaction       = pl.col('tbp_lv_norm_border') * pl.col('tbp_lv_norm_color'),
            border_color_interaction_2     = pl.col('tbp_lv_norm_border') * pl.col('tbp_lv_norm_color') / (pl.col('tbp_lv_norm_border') + pl.col('tbp_lv_norm_color')),
            size_color_contrast_ratio      = pl.col('clin_size_long_diam_mm') / pl.col('tbp_lv_deltaLBnorm'),
            age_normalized_nevi_confidence = pl.col('tbp_lv_nevi_confidence') / pl.col('age_approx'),
            age_normalized_nevi_confidence_2 = (pl.col('clin_size_long_diam_mm')**2 + pl.col('age_approx')**2).sqrt(),
            color_asymmetry_index          = pl.col('tbp_lv_radial_color_std_max') * pl.col('tbp_lv_symm_2axis'),
        )
        .with_columns(
            volume_approximation_3d        = pl.col('tbp_lv_areaMM2') * (pl.col('tbp_lv_x')**2 + pl.col('tbp_lv_y')**2 + pl.col('tbp_lv_z')**2).sqrt(),
            color_range                    = (pl.col('tbp_lv_L') - pl.col('tbp_lv_Lext')).abs() + (pl.col('tbp_lv_A') - pl.col('tbp_lv_Aext')).abs() + (pl.col('tbp_lv_B') - pl.col('tbp_lv_Bext')).abs(),
            shape_color_consistency        = pl.col('tbp_lv_eccentricity') * pl.col('tbp_lv_color_std_mean'),
            border_length_ratio            = pl.col('tbp_lv_perimeterMM') / (2 * np.pi * (pl.col('tbp_lv_areaMM2') / np.pi).sqrt()),
            age_size_symmetry_index        = pl.col('age_approx') * pl.col('clin_size_long_diam_mm') * pl.col('tbp_lv_symm_2axis'),
            index_age_size_symmetry        = pl.col('age_approx') * pl.col('tbp_lv_areaMM2') * pl.col('tbp_lv_symm_2axis'),
        )
        # .with_columns(
        #     ((pl.col(col) - pl.col(col).mean().over('patient_id')) / (pl.col(col).std().over('patient_id') + err)).alias(f'{col}_patient_norm') for col in (num_cols + new_num_cols)
        # )
        .with_columns(
            count_per_patient = pl.col('isic_id').count().over('patient_id'),
        )
        .with_columns(
            pl.col(cat_cols).cast(pl.Categorical),
        )
        .to_pandas()
        # .set_index(id_col)
    )

In [ ]:
# encoding categorical features for models that can't handle is as is. 
def preprocess(df_train, df_test):
    global cat_cols
    
    encoder = OneHotEncoder(sparse_output=False, dtype=np.int32, handle_unknown='ignore')
    encoder.fit(df_train[cat_cols])
    
    new_cat_cols = [f'onehot_{i}' for i in range(len(encoder.get_feature_names_out()))]

    df_train[new_cat_cols] = encoder.transform(df_train[cat_cols])
    df_train[new_cat_cols] = df_train[new_cat_cols].astype('category')

    df_test[new_cat_cols] = encoder.transform(df_test[cat_cols])
    df_test[new_cat_cols] = df_test[new_cat_cols].astype('category')

    for col in cat_cols:
        feature_cols.remove(col)

    feature_cols.extend(new_cat_cols)
    cat_cols = new_cat_cols
    
    return df_train, df_test

### 4.3 Custom metric

In [ ]:
def custom_metric(estimator, X, y_true):
    y_hat = estimator.predict_proba(X)[:, 1]
    min_tpr = 0.80
    max_fpr = abs(1 - min_tpr)
    
    v_gt = abs(y_true - 1)
    v_pred = np.array([1.0 - x for x in y_hat])
    
    partial_auc_scaled = roc_auc_score(v_gt, v_pred, max_fpr=max_fpr)
    partial_auc = 0.5 * max_fpr**2 + (max_fpr - 0.5 * max_fpr**2) / (1.0 - 0.5) * (partial_auc_scaled - 0.5)
    
    return partial_auc

### 4.4 Apply preprocessing

In [ ]:
# .with_columns(
#     ((pl.col(col) - pl.col(col).mean().over('patient_id')) / (pl.col(col).std().over('patient_id') + err)).alias(f'{col}_patient_norm') for col in (num_cols + new_num_cols)
# )

# preprocess
df_train = read_data(train_meta)
new_columns = {}
for col in num_cols + new_num_cols:
    patient_mean = df_train.groupby('patient_id')[col].transform('mean')
    patient_std = df_train.groupby('patient_id')[col].transform('std')
    # Store the normalized column in the dictionary
    new_columns[f'{col}_patient_norm'] = (df_train[col] - patient_mean) / (patient_std + err)
df_train = pd.concat([df_train, pd.DataFrame(new_columns)], axis=1)


# preprocess
df_test = read_data(test_meta)
new_columns = {}
for col in num_cols + new_num_cols:
    patient_mean_test = df_test.groupby('patient_id')[col].transform('mean')
    patient_std_test = df_test.groupby('patient_id')[col].transform('std')
    # Store the normalized column in the dictionary
    new_columns[f'{col}_patient_norm'] = (df_test[col] - patient_mean_test) / (patient_std_test + err)
df_test = pd.concat([df_test, pd.DataFrame(new_columns)], axis=1)

df_subm = pd.read_csv(CFG.SAMPLE_SUB, index_col=id_col)

# dealing categorical data
df_train, df_test = preprocess(df_train, df_test)

train_meta = df_train
test_meta = df_test

Check the df

In [ ]:
print(train_meta.shape)
print(test_meta.shape)

display(train_meta.head())
display(test_meta.head())

# 5. Dataset

In [ ]:
# setting seed in each env
def set_random_seed(seed: int = 42, deterministic: bool = False):
    """Set seeds"""
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = deterministic  # type: ignore

# function to set tensor to device
def to_device(
    tensors: tp.Union[tp.Tuple[torch.Tensor], tp.Dict[str, torch.Tensor]],
    device: torch.device, *args, **kwargs
):
    if isinstance(tensors, tuple):
        return (t.to(device, *args, **kwargs) for t in tensors)
    elif isinstance(tensors, dict):
        return {
            k: t.to(device, *args, **kwargs) for k, t in tensors.items()}
    else:
        return tensors.to(device, *args, **kwargs)

In [ ]:

def get_transforms(include_light=False):
    augmentations_train = A.Compose([
    A.Transpose(p=0.5),
    A.VerticalFlip(p=0.5),
    A.HorizontalFlip(p=0.5),
    A.ColorJitter(brightness=0.2, contrast=0.2, p=0.3),
    A.OneOf([
            A.MotionBlur(blur_limit=5, p=0.5),
            A.MedianBlur(blur_limit=5, p=0.5),
            A.GaussianBlur(blur_limit=5, p=0.5),
        ], p=0.2),
    A.GaussNoise(var_limit=(5.0, 30.0), p=0.1),
    # A.OpticalDistortion(distort_limit=0.5, p=0.3),
    # A.GridDistortion(num_steps=5, distort_limit=0.5, p=0.3),
    # A.ElasticTransform(alpha=1, p=0.3),
    # A.CLAHE(clip_limit=2.0, p=0.3),
    # A.HueSaturationValue(hue_shift_limit=5, sat_shift_limit=10, val_shift_limit=5, p=0.3),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, border_mode=0, p=0.3),
    A.CoarseDropout(max_holes=20, min_holes=10, p=0.3),
    A.Resize(CFG.img_size, CFG.img_size),
    # A.Normalize(
    #                 mean=[0.485, 0.456, 0.406], 
    #                 std=[0.229, 0.224, 0.225], 
    #                 max_pixel_value=255.0, 
    #                 p=1.0
    #             ),
    ToTensorV2(p=1)
    ])
    augmentations_train_light = A.Compose([
        A.Transpose(p=0.5),
        A.VerticalFlip(p=0.5),
        A.HorizontalFlip(p=0.5),
        A.ColorJitter(brightness=0.2, contrast=0.2, p=0.1),
        A.OneOf([
            A.MotionBlur(blur_limit=5, p=0.5),
            A.MedianBlur(blur_limit=5, p=0.5),
            A.GaussianBlur(blur_limit=5, p=0.5),
        ], p=0.1),
        # A.GaussNoise(var_limit=(5.0, 30.0), p=0.5),
        # A.OpticalDistortion(distort_limit=0.5, p=0.5),
        # A.GridDistortion(num_steps=5, distort_limit=0.5, p=0.5),
        # A.ElasticTransform(alpha=1, p=0.5),
        # A.CLAHE(clip_limit=2.0, p=0.5),
        # A.HueSaturationValue(hue_shift_limit=5, sat_shift_limit=10, val_shift_limit=5, p=0.5),
        A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, border_mode=0, p=0.2),
        A.CoarseDropout(max_holes=30, min_holes=20, p=0.2),
        A.Resize(CFG.img_size, CFG.img_size),
        # A.Normalize(
        #                 mean=[0.485, 0.456, 0.406], 
        #                 std=[0.229, 0.224, 0.225], 
        #                 max_pixel_value=255.0, 
        #                 p=1.0
        #             ),
        ToTensorV2(p=1)
    ])
    
    augmentations_test = A.Compose([
        A.Resize(CFG.img_size, CFG.img_size),
        # A.Normalize(
        #                 mean=[0.485, 0.456, 0.406], 
        #                 std=[0.229, 0.224, 0.225], 
        #                 max_pixel_value=255.0, 
        #                 p=1.0
        #             ),
        ToTensorV2(p=1)
    ])
    if include_light:
        return augmentations_train, augmentations_test, augmentations_train_light
    return augmentations_train, augmentations_test

# def get_transforms():
#     data_transforms = {
#         "train": A.Compose([
#             A.Resize(CFG.img_size, CFG.img_size),
#             A.RandomRotate90(p=0.5),
#             A.Flip(p=0.5),
#             A.Downscale(p=0.25),
#             A.ShiftScaleRotate(shift_limit=0.1, 
#                             scale_limit=0.15, 
#                             rotate_limit=60, 
#                             p=0.5),
#             A.HueSaturationValue(
#                     hue_shift_limit=0.2, 
#                     sat_shift_limit=0.2, 
#                     val_shift_limit=0.2, 
#                     p=0.5
#                 ),
#             A.RandomBrightnessContrast(
#                     brightness_limit=(-0.1,0.1), 
#                     contrast_limit=(-0.1, 0.1), 
#                     p=0.5
#                 ),
#             A.Normalize(
#                     mean=[0.485, 0.456, 0.406], 
#                     std=[0.229, 0.224, 0.225], 
#                     max_pixel_value=255.0, 
#                     p=1.0
#                 ),
#             ToTensorV2()], p=1.),
        
#         "valid": A.Compose([
#             A.Resize(CFG.img_size, CFG.img_size),
#             A.Normalize(
#                     mean=[0.485, 0.456, 0.406], 
#                     std=[0.229, 0.224, 0.225], 
#                     max_pixel_value=255.0, 
#                     p=1.0
#                 ),
#             ToTensorV2()], p=1.)
#     }
#     return data_transforms["train"], data_transforms["valid"]

# def get_transforms():
#     train_transforms = A.Compose([
#         A.Transpose(p=0.5),
#         A.VerticalFlip(p=0.5),
#         A.HorizontalFlip(p=0.5),
#         A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.75),
#         A.OneOf([
#             A.MotionBlur(blur_limit=5),
#             A.MedianBlur(blur_limit=5),
#             A.GaussianBlur(blur_limit=5),
#             A.GaussNoise(var_limit=(5.0, 30.0)),
#         ], p=0.7),

#         A.OneOf([
#             A.OpticalDistortion(distort_limit=1.0),
#             A.GridDistortion(num_steps=5, distort_limit=1.),
#             A.ElasticTransform(alpha=3),
#         ], p=0.7),

#         A.CLAHE(clip_limit=4.0, p=0.7),
#         A.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=20, val_shift_limit=10, p=0.5),
#         A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, border_mode=0, p=0.85),
#         A.Resize(CFG.img_size, CFG.img_size),
#         A.CoarseDropout(max_height=int(CFG.img_size * 0.375), max_width=int(CFG.img_size * 0.375), max_holes=1, min_holes=1, p=0.7),    
#         A.Normalize()
#     ])

#     val_transforms = A.Compose([
#         A.Resize(CFG.img_size, CFG.img_size),
#         A.Normalize()
#     ])

#     return train_transforms, val_transforms


In [ ]:

class ISICDataset(Dataset):
    def __init__(self, 
                df: pd.DataFrame,
                fp_hdf: str|Path,
                transform: A.Compose=None,
                ):
        self.df = df
        if 'target' in self.df.columns:
            self.is_training = True
            self.targets = df['target'].values
        else:
            self.is_training = False
        self.fp_hdf = h5py.File(fp_hdf, mode="r")
        self.isic_ids = df['isic_id'].values
        self.transform = transform
        
    def __len__(self):
        return len(self.isic_ids)
    
    def __getitem__(self, index):
        isic_id = self.isic_ids[index]
        image = np.array(Image.open(BytesIO(self.fp_hdf[isic_id][()])))
        if self.is_training:
            target = self.targets[index]
        else:
            target = []
        
        if CFG.AUX_LOSS:
            pass
        return (self._apply_transform(image), target)
    
    def _apply_transform(self, img:np.ndarray):
        """apply transform to image"""
        if self.transform:
            transformed = self.transform(image=img)
            img = transformed["image"]# .float()# .half()
        return img

# 6. Image Model

In [ ]:

def F_rgb2hsv(rgb: torch.Tensor) -> torch.Tensor:
    cmax, cmax_idx = torch.max(rgb, dim=1, keepdim=True)
    cmin = torch.min(rgb, dim=1, keepdim=True)[0]
    delta = cmax - cmin
    hsv_h = torch.empty_like(rgb[:, 0:1, :, :])
    cmax_idx[delta == 0] = 3
    hsv_h[cmax_idx == 0] = (((rgb[:, 1:2] - rgb[:, 2:3]) / delta) % 6)[cmax_idx == 0]
    hsv_h[cmax_idx == 1] = (((rgb[:, 2:3] - rgb[:, 0:1]) / delta) + 2)[cmax_idx == 1]
    hsv_h[cmax_idx == 2] = (((rgb[:, 0:1] - rgb[:, 1:2]) / delta) + 4)[cmax_idx == 2]
    hsv_h[cmax_idx == 3] = 0.
    hsv_h /= 6.
    hsv_s = torch.where(cmax == 0, torch.tensor(0.).type_as(rgb), delta / cmax)
    hsv_v = cmax
    return torch.cat([hsv_h, hsv_s, hsv_v], dim=1)

# linear can treat any input size
class DynamicLinear(nn.Module):
    def __init__(self, out_size):
        super(DynamicLinear, self).__init__()
        self.out_size = out_size
        self.linear = None

    def forward(self, x):
        if self.linear is None:
            in_features = x.size(-1)
            self.linear = nn.Linear(in_features, self.out_size).to(x.device)
        
        return self.linear(x)


class timmModel(nn.Module):

    def __init__(
            self,
            model_name: str,
            pretrained: bool,
            in_channels: int,
            num_classes: int,
            is_training: bool=False
        ):
        super().__init__()
        if 'eva' in CFG.model_name:
            self.model = timm.create_model(
                model_name=model_name, 
                pretrained=pretrained, 
                in_chans=in_channels,
                num_classes=num_classes,
                # global_pool=''
            )
        else:
            self.model = timm.create_model(
                model_name=model_name, 
                pretrained=pretrained, 
                in_chans=in_channels,
                num_classes=num_classes,
                global_pool=''
            )
        
        self.output_type = ['infer', 'loss']
        self.is_training = is_training

        # model output dim
        dim = CFG.output_dim_models[CFG.model_name]
        
        self.dropout = nn.ModuleList([
            nn.Dropout(0.5) for i in range(5)
        ])
        # self.target=nn.Linear(dim, 1)
        self.target=DynamicLinear(out_size=1)
        # self.target_aux=nn.Linear(dim, 1)
        

    def forward(self, x, target=None):
        batch_size = len(x)
        
        x = torch.cat([x, F_rgb2hsv(x)],1) # (bs, dim, h, w)
        x = self.model(x)    
        # print('model output: ', x.shape)
        
        if not 'eva' in CFG.model_name:
            pool = F.adaptive_avg_pool2d(x, 1)
            # print('after pooling: ', pool.shape)
            reshaped = pool.reshape(batch_size, -1)  
            # print('after reshape: ', reshaped.shape)
        
        else:
            reshaped = x
        
        if self.is_training:
            logit = 0
            # print(len(self.dropout))
            for i in range(len(self.dropout)):
                dropped = self.dropout[i](reshaped)
                # print(dropped.shape)
                logit += self.target(dropped)
            logit = logit / len(self.dropout)
        else:
            logit = self.target(reshaped)
            
        output = {}
        if 'loss' in self.output_type:
            if target.dim() == 1:
                target = target.view(-1, 1)
            output['bce_loss'] = F.binary_cross_entropy_with_logits(logit.float(), target.float())

        if 'infer' in self.output_type:
            output['target'] = torch.sigmoid(logit.float())

        return output
    
    def initialize_dummy(self):
        # Initialize the DynamicLinear layer with dummy input data
        original_output_type = self.output_type
        self.output_type = ['infer']
        
        dummy_input = torch.zeros(1, 3, CFG.img_size, CFG.img_size)  # Assuming input size (1, 3, 224, 224)
        _ = self.forward(dummy_input)
        
        self.output_type = original_output_type

In [ ]:
print(train_meta.columns)
display(train_meta.head())

In [ ]:
if not CFG.is_infer:


    if CFG.view:
        def show_batch(ds, row=3, col=3, color='rgb'):
            fig = plt.figure(figsize=(10, 10))
            img_index = np.random.randint(0, len(ds)-1, row*col)
            
            for i in range(len(img_index)):
                img, label = ds[img_index[i]]
                img_rgb = img[:3, :, :]
                # img_hsv = img[3:6, :, :]
                
                if color=='rgb':
                    img = img_rgb
                # elif color=='hsv':
                #     img = img_hsv
                
                if isinstance(img, torch.Tensor):
                    img = img.detach().numpy()
                    # Check if the image is in (C, H, W) and transpose it to (H, W, C)
                    if img.shape[0] == 3:  # Assuming the image is (C, H, W)
                        img = np.transpose(img, (1, 2, 0))
                
                ax = fig.add_subplot(row, col, i + 1, xticks=[], yticks=[])
                ax.imshow(img)  # Remove cmap parameter for RGB images
                ax.set_title(f'ID: {img_index[i]}; Target: {label}')
            
            plt.tight_layout()
            plt.show()

        _train_transform, _ = get_transforms()
        if CFG.alldata_isic_archive:
            _dataset = ISICDataset(df=train_meta, fp_hdf=CFG.TRAIN_HDF5_COMBINED, transform=_train_transform)
        else:
            _dataset = ISICDataset(df=train_meta, fp_hdf=CFG.TRAIN_HDF5, transform=_train_transform)
        show_batch(_dataset)
        # show_batch(_dataset, color='hsv')


# 7. Training

In [ ]:
if not CFG.is_infer:


    # reference
    def train_one_fold(val_fold: int, 
                    train: pd.DataFrame,
                    output_path: str|Path
                    ):
        """Main"""
        # If True, forces cuDNN to benchmark multiple convolution algorithms and choose the fastest one
        torch.backends.cudnn.benchmark = True
        set_random_seed(CFG.random_seed, deterministic=CFG.deterministic)
        # set device with pytorch env
        device = torch.device(CFG.device)
        # print(device)
        
        train_transform, val_transform = get_transforms()
        
    #     train_dataset = Bird2024Dataset(**train_path_label, transform=train_transform)
        if CFG.alldata_isic_archive:
            train_dataset = ISICDataset(train[train['fold']!=val_fold], CFG.TRAIN_HDF5_COMBINED, train_transform)
            train_dataset_noaugment = ISICDataset(train[train['fold']!=val_fold], CFG.TRAIN_HDF5_COMBINED, val_transform)
            val_dataset = ISICDataset(train[train['fold']==val_fold], CFG.TRAIN_HDF5_COMBINED, val_transform)
        else:
            train_dataset = ISICDataset(train[train['fold']!=val_fold], CFG.TRAIN_HDF5, train_transform)
            train_dataset_noaugment = ISICDataset(train[train['fold']!=val_fold], CFG.TRAIN_HDF5, val_transform)
            val_dataset = ISICDataset(train[train['fold']==val_fold], CFG.TRAIN_HDF5, val_transform)
        
        train_loader = torch.utils.data.DataLoader(
            train_dataset, batch_size=CFG.batch_size, num_workers=4, shuffle=True, drop_last=True)
        train_loader_noaugment = torch.utils.data.DataLoader(
            train_dataset_noaugment, batch_size=CFG.batch_size, num_workers=4, shuffle=True, drop_last=True)
        
        val_loader = torch.utils.data.DataLoader(
            val_dataset, batch_size=CFG.batch_size, num_workers=4, shuffle=False, drop_last=False)
        
        model = timmModel(
            model_name=CFG.model_name, 
            pretrained=CFG.pretrained, 
            num_classes=0, # no classification head
            in_channels=6, # RBG+HSV
            is_training=True
        )
        # transfer learning
        if CFG.PRETRAINED_MODEL:
            model.initialize_dummy()
            model.load_state_dict(torch.load(CFG.PRETRAINED_MODEL, map_location=device))
            
        model = model.to(device)

        
        optimizer = optim.AdamW(params=model.parameters(), lr=CFG.lr, weight_decay=CFG.weight_decay)
        scheduler = lr_scheduler.OneCycleLR(
            optimizer=optimizer, epochs=CFG.max_epoch,
            pct_start=0.0, steps_per_epoch=len(train_loader),
            max_lr=CFG.lr, div_factor=25, final_div_factor=1e4
        )
        
    #     loss_func = KLDivLossWithLogits()
        # loss_func = nn.CrossEntropyLoss()
        # loss_func = nn.BCEWithLogitsLoss()
    #     loss_func = FocalLossBCE()
        # loss_func.to(device)
    #     loss_func_val = KLDivLossWithLogitsForVal()
    #     loss_func_val = nn.CrossEntropyLoss()
        # loss_func_val = nn.BCEWithLogitsLoss()
    #     loss_func_val = FocalLossBCE()
        
        use_amp = CFG.enable_amp
        scaler = amp.GradScaler('cuda', enabled=use_amp)
        # scaler = torch.cuda.amp.GradScaler(enabled=use_amp)
        
        best_val_loss = 1.0e+09
        best_epoch = 0
        train_loss = 0
        val_loss = 0
        cumulative_elapsed_time = 0
        
        for epoch in range(1, CFG.max_epoch + 1):
            if CFG.change_dataset & epoch >= ((CFG.max_epoch+1) * 0.6):
                train_loader = train_loader_noaugment
            epoch_start = time()
            model.train()
            for batch in tqdm(train_loader):
                
                x, t = batch
                if CFG.standardization:
                    x = (x - x.min()) / (x.max() - x.min() +1e-6) * 255
                else:
                    x = x.float()/255
                # t = t.float()
    #             print(x)
    #             print(t)
                x = to_device(x, device)
                t = to_device(t, device)
                    
                optimizer.zero_grad()
                # with torch.cuda.amp.autocast(use_amp):
                with amp.autocast('cuda', enabled=use_amp):
                    output = model(x, t)
                    loss = output['bce_loss']
                
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
                train_loss += loss.item()
                scheduler.step()
                
            train_loss /= len(train_loader)
                
            model.eval()
            for batch in tqdm(val_loader):
                x, t = batch
                if CFG.standardization:
                    x = (x - x.min()) / (x.max() - x.min() +1e-6) * 255
                else:
                    x = x.float()/255
                x = to_device(x, device)
                t = to_device(t, device)
                # with torch.no_grad(), torch.cuda.amp.autocast(use_amp):
                with torch.no_grad(), amp.autocast('cuda', enabled=use_amp):
                    output = model(x, t)
                loss = output['bce_loss']
                val_loss += loss.item()
            val_loss /= len(val_loader)
            
            if val_loss < best_val_loss:
                best_epoch = epoch
                best_val_loss = val_loss
                # print("save model")
                torch.save(model.state_dict(), str(output_path / f'snapshot_epoch_{epoch}.pth'))
            
            elapsed_time = time() - epoch_start
            cumulative_elapsed_time += elapsed_time
            with open(str(output_path / f'train_log.txt'), 'a') as f:
                f.write('\n')
                f.write(f'exp: {CFG.exp}, [epoch {epoch}] train loss: {train_loss: .6f}, val loss: {val_loss: .6f}, elapsed_time: {elapsed_time: .3f}')
            
            # log
            if CFG.enable_wandb:
                wandb.log({"Train Loss": train_loss, 
                        "Val Loss": val_loss, 
                        "Cumulative Time(m)": cumulative_elapsed_time/60,
                        "Fold": val_fold}
                        )
            print(
                f"[epoch {epoch}] train loss: {train_loss: .6f}, val loss: {val_loss: .6f}, cumulative_elapsed_time: {cumulative_elapsed_time/60: .1f}m , elapsed_time: {elapsed_time/60: .3f}m")
            
            
            
            if epoch - best_epoch > CFG.es_patience:
                with open(str(output_path / f'train_log.txt'), 'a') as f:
                    f.write('\n')
                    f.write("Early Stopping!")
                print("Early Stopping!")
                break
                
            train_loss = 0
            val_loss = 0
                
        return val_fold, best_epoch, best_val_loss

### delete previous exp data

In [ ]:
if not CFG.is_infer:


    # select the best model and delete others
    best_log_list = []
    for (fold_id) in range(CFG.n_folds):
        
        # select the best model
        exp_dir_path = CFG.OUTPUT_DIR / f"fold{fold_id}"        
        for p in exp_dir_path.glob("*.pth"):
            # delete
            p.unlink()
        for p in exp_dir_path.glob("*.txt"):
            # delete
            p.unlink()

### Train  

In [ ]:
if not CFG.is_infer:

    score_list = []
    print('='*20)
    print('experiment: ', CFG.exp)
    print('='*20)
    for fold_id in range(CFG.n_folds):
        output_path = CFG.OUTPUT_DIR / f"fold{fold_id}"
        output_path.mkdir(exist_ok=True, parents=True)
        print(f"[fold{fold_id}]")
        score_list.append(train_one_fold(fold_id, train_meta, output_path))
        
    with open(CFG.OUTPUT_LOG, 'a') as f:
        f.write('\n')
        f.write(f'exp: {CFG.exp}')
        f.write('\n')
        f.write(str(score_list))

# 8. Validation

In [ ]:
if not CFG.is_infer:

    print(score_list)
    score_list_log = score_list

In [ ]:
if not CFG.is_infer:


    # select the best model and delete others
    best_log_list = []
    for (fold_id, best_epoch, _) in score_list:
        
        # select the best model
        exp_dir_path = CFG.OUTPUT_DIR / f"fold{fold_id}"
        best_model_path = exp_dir_path / f"snapshot_epoch_{best_epoch}.pth"
        # copy to new place
        copy_to = CFG.OUTPUT_DIR / f"./best_model_fold{fold_id}.pth"
        shutil.copy(best_model_path, copy_to)
        
        for p in exp_dir_path.glob("*.pth"):
            # delete
            p.unlink()
        # for p in exp_dir_path.glob("*.txt"):
        #     # delete
        #     p.unlink()

In [ ]:
# Function for inference
def run_inference_loop(model, loader, device):
    model.to(device)
    model.eval()
    model.output_type = ['infer']
    pred_list = []
    with torch.no_grad():
        for batch in tqdm(loader):
            x = to_device(batch[0], device)
            if CFG.standardization:
                x = (x - x.min()) / (x.max() - x.min() +1e-6) * 255
            else:
                x = x.float()/255
            output = model(x)
            y = output['target']
            pred_list.append(y.detach().cpu().numpy())
    
    # concatenate to vertical (to df like from long scroll like)
    pred_arr = np.concatenate(pred_list)
    del pred_list
    return pred_arr

### Prediction

In [ ]:
if not CFG.is_infer:


    # predict for train data with metrix(CV. not test data)


    # duplicate check
    column_to_check = 'isic_id'
    # Drop duplicate rows based on the specified column
    train_meta = train_meta.drop_duplicates(subset=[column_to_check], keep='first').reset_index(drop=True)

    # label_arr = train[CLASSES].values
    oof_pred_arr = np.zeros((len(train_meta), CFG.n_classes-1))
    if CFG.TTA:
        oof_pred_arr_TTA = np.zeros((len(train_meta), CFG.n_classes-1))
    score_list = []

    for fold_id in range(CFG.n_folds):
        print(f"\n[fold {fold_id}]")
        device = torch.device(CFG.device)

        # get_dataloader
        val_transform_TTA, val_transform = get_transforms()
        if CFG.alldata_isic_archive:
            val_dataset = ISICDataset(df=train_meta[train_meta["fold"] == fold_id], fp_hdf=CFG.TRAIN_HDF5_COMBINED, transform=val_transform)
            val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=CFG.batch_size, num_workers=4, shuffle=False, drop_last=False)
            if CFG.TTA:
                val_dataset_TTA = ISICDataset(df=train_meta[train_meta["fold"] == fold_id], fp_hdf=CFG.TRAIN_HDF5_COMBINED, transform=val_transform_TTA)
                val_loader_TTA = torch.utils.data.DataLoader(val_dataset_TTA, batch_size=CFG.batch_size, num_workers=4, shuffle=False, drop_last=False)
        else:
            val_dataset = ISICDataset(df=train_meta[train_meta["fold"] == fold_id], fp_hdf=CFG.TRAIN_HDF5, transform=val_transform)
            val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=CFG.batch_size, num_workers=4, shuffle=False, drop_last=False)
            if CFG.TTA:
                val_dataset_TTA = ISICDataset(df=train_meta[train_meta["fold"] == fold_id], fp_hdf=CFG.TRAIN_HDF5, transform=val_transform_TTA)
                val_loader_TTA = torch.utils.data.DataLoader(val_dataset_TTA, batch_size=CFG.batch_size, num_workers=4, shuffle=False, drop_last=False)

        # # get model
        model_path = CFG.OUTPUT_DIR / f"best_model_fold{fold_id}.pth"
        model = timmModel(
            model_name=CFG.model_name, 
            pretrained=False, 
            in_channels=6,
            num_classes=0,
            is_training=False
        )
        model.initialize_dummy()  # Initialize with dummy data for dynamic linear
        model.load_state_dict(torch.load(model_path, map_location=device))

        # # inference
        val_pred = run_inference_loop(model, val_loader, device)
        val_idx = train_meta[train_meta["fold"] == fold_id].index.values
        oof_pred_arr[val_idx] = val_pred
        if CFG.TTA:
            val_pred_TTA = run_inference_loop(model, val_loader_TTA, device)
            val_idx_TTA = train_meta[train_meta["fold"] == fold_id].index.values
            oof_pred_arr_TTA[val_idx_TTA] = val_pred_TTA

        del val_idx
        del model, val_loader
        torch.cuda.empty_cache()
        gc.collect()
    


### Merging

In [ ]:
if not CFG.is_infer:
    if CFG.TTA:
        # ref. CFG.TTA_rate = {'None':value, 'with_train_aug':value}
        oof_pred_arr_merged = (oof_pred_arr * CFG.TTA_rate['None']) + (oof_pred_arr_TTA * CFG.TTA_rate['with_train_aug'])

### test of averaged model

In [ ]:
# test of averaged model
if CFG.test_of_averaged_model:
    # predict for train data with metrix(CV. not test data)


    # duplicate check
    column_to_check = 'isic_id'
    # Drop duplicate rows based on the specified column
    train_meta = train_meta.drop_duplicates(subset=[column_to_check], keep='first').reset_index(drop=True)

    # label_arr = train[CLASSES].values
    oof_pred_arr = np.zeros((len(train_meta), CFG.n_classes-1))
    score_list = []

    for fold_id in range(1):
        print(f"\n[fold {fold_id}]")
        device = torch.device(CFG.device)

        # get_dataloader
        _, val_transform = get_transforms()
        val_dataset = ISICDataset(df=train_meta, fp_hdf=CFG.TRAIN_HDF5_COMBINED, transform=val_transform)
        val_loader = torch.utils.data.DataLoader(
            val_dataset, batch_size=CFG.batch_size, num_workers=4, shuffle=False, drop_last=False)

        # # get model
        model_path = CFG.OUTPUT_DIR / f"averaged_model.pth"
        model = timmModel(
            model_name=CFG.model_name, 
            pretrained=False, 
            in_channels=6,
            num_classes=0,
            is_training=False
        )
        model.initialize_dummy()  # Initialize with dummy data for dynamic linear
        model.load_state_dict(torch.load(model_path, map_location=device))

        # # inference
        val_pred = run_inference_loop(model, val_loader, device)
        # val_idx = train_meta[train_meta["fold"] == fold_id].index.values
        oof_pred_arr = val_pred

        del val_idx
        del model, val_loader
        torch.cuda.empty_cache()
        gc.collect()

### Metrix

In [ ]:
def comp_score(solution: pd.DataFrame, submission: pd.DataFrame, row_id_column_name: str, min_tpr: float=0.80):
    v_gt = abs(np.asarray(solution.values)-1)
    v_pred = np.array([1.0 - x for x in submission.values])
    max_fpr = abs(1-min_tpr)
    partial_auc_scaled = roc_auc_score(v_gt, v_pred, max_fpr=max_fpr)
    # change scale from [0.5, 1.0] to [0.5 * max_fpr**2, max_fpr]
    # https://math.stackexchange.com/questions/914823/shift-numbers-into-a-different-range
    partial_auc = 0.5 * max_fpr**2 + (max_fpr - 0.5 * max_fpr**2) / (1.0 - 0.5) * (partial_auc_scaled - 0.5)
    return partial_auc


### CV Score

In [ ]:
if not CFG.is_infer:

    # make true array
    label_arr = train_meta['target']
    # one-hot
    # ture_arr = np.zeros((label_arr.size, CFG.n_classes))
    # ture_arr[np.arange(label_arr.size), label_arr] = 1
    ture_arr = pd.DataFrame(label_arr)

    # oof
    oof = pd.DataFrame(oof_pred_arr)
    if CFG.TTA:
        oof = pd.DataFrame(oof_pred_arr_merged)

    micro_roc_auc_ovr = comp_score(
        ture_arr,
        oof,
        ""
    )

    print(f"CV: Micro-averaged One-vs-Rest ROC AUC score:\n{micro_roc_auc_ovr:.10f}")
    if CFG.enable_wandb:
        wandb.log({"CV": micro_roc_auc_ovr})


In [ ]:
if not CFG.is_infer:

    with open(CFG.OUTPUT_LOG, 'a') as f:
        f.write('\n')
        f.write(f'exp: {CFG.exp}')
        f.write('\n')
        f.write(f"CV: Micro-averaged One-vs-Rest ROC AUC score: {micro_roc_auc_ovr:.10f}")

In [ ]:
if not CFG.is_infer:

    display(oof.head())
    display(ture_arr.head())
    display(oof.tail())
    display(ture_arr.tail())

# 9. Inference

In [ ]:
# predict for test data with metrix
if (CFG.is_kaggle & CFG.is_infer) or CFG.use_tabler:

    pred_arr = []
    pred_arr_TTA = []

    for fold_id in range(CFG.n_folds):
        print(f"\n[fold {fold_id}]")
        device = torch.device(CFG.device)

        # get_dataloader
        val_transform_TTA, val_transform = get_transforms()
        val_dataset = ISICDataset(df=test_meta, fp_hdf=CFG.TEST_HDF5, transform=val_transform)
        val_loader = torch.utils.data.DataLoader(
            val_dataset, batch_size=CFG.batch_size, num_workers=2, shuffle=False, drop_last=False)
        if CFG.TTA:
            val_dataset_TTA = ISICDataset(df=test_meta, fp_hdf=CFG.TEST_HDF5, transform=val_transform_TTA)
            val_loader_TTA = torch.utils.data.DataLoader(
                val_dataset_TTA, batch_size=CFG.batch_size, num_workers=2, shuffle=False, drop_last=False)

        # get model
        model_path = CFG.OUTPUT_DIR / f"best_model_fold{fold_id}.pth"
        model = timmModel(
            model_name=CFG.model_name, 
            pretrained=False, 
            in_channels=6,
            num_classes=0,
            is_training=False
        )
        model.initialize_dummy()
        model.load_state_dict(torch.load(model_path, map_location=device))

        # # inference
        val_pred = run_inference_loop(model, val_loader, device)
        pred_arr.append(val_pred)
        if CFG.TTA:
            val_pred_TTA = run_inference_loop(model, val_loader_TTA, device)
            pred_arr_TTA.append(val_pred_TTA)

        del model, val_loader, val_pred
        if CFG.TTA:
            del val_loader_TTA, val_pred_TTA
        torch.cuda.empty_cache()
        gc.collect()
        
    # averaging
    print(len(pred_arr))
    if len(pred_arr) >= 2:    
        pred_arr = np.mean(pred_arr, axis=0)
    print(pred_arr.shape)
    
    if CFG.TTA:
        print(len(pred_arr_TTA))
        if len(pred_arr_TTA) >= 2:    
            pred_arr_TTA = np.mean(pred_arr_TTA, axis=0)
        print(pred_arr_TTA.shape)

        pred_arr = (pred_arr * CFG.TTA_rate['None']) + (pred_arr_TTA * CFG.TTA_rate['with_train_aug'])


    pred_df = pd.DataFrame(pred_arr, columns=['target'])
    display(pred_df)

    # submission
    df_sub = pd.read_csv(CFG.SAMPLE_SUB)
    df_sub["target"] = pred_df
    display(df_sub.head())
    df_sub.to_csv('submission.csv', index=False)
 
    


# 10. Tabler

In [ ]:
if CFG.use_tabler:
    image_pred_col = 'image_prediction'
    
    if not CFG.is_infer:
        # add image model prediction to train_meta as a feature
        
        oof = pd.DataFrame(oof_pred_arr, columns=[image_pred_col])
        display(oof.head())
        assert len(oof) == len(train_meta)
        train_meta[image_pred_col] = oof[image_pred_col].values
        # train_meta = train_meta.merge(oof[image_pred_col], 
        #                       on='isic_id', 
        #                       how='left')
        
    # add using feature
    if CFG.image_pred_as_tabler_feature:
        feature_cols.append(image_pred_col)

    # add image model prediction to test_meta as a feature
    assert len(pred_df) == len(test_meta)
    display(pred_df)
    test_meta[image_pred_col] = pred_df['target'].values
    # test_meta = test_meta.merge(pred_df[image_pred_col], 
    #                       on='isic_id', 
    #                       how='left')
    # already added  
    # feature_cols.append(image_pred_col)

### 10.1 Optuna HyperParam Tuned Models

##### 10.1.1 LGBM

In [ ]:
if CFG.use_tabler and ((not CFG.is_infer) or (not CFG.image_pred_as_tabler_feature)):
    lgb_params = {
        'objective':        'binary',
        'verbosity':        -1,
        'n_iter':           200,
        'boosting_type':    'gbdt',
        'random_state':     seed,
        'lambda_l1':        0.08758718919397321, 
        'lambda_l2':        0.0039689175176025465, 
        'learning_rate':    0.03231007103195577, 
        'max_depth':        4, 
        'num_leaves':       103, 
        'colsample_bytree': 0.8329551585827726, 
        'colsample_bynode': 0.4025961355653304, 
        'bagging_fraction': 0.7738954452473223, 
        'bagging_freq':     4, 
        'min_data_in_leaf': 85, 
        'scale_pos_weight': 2.7984184778875543,
    }

    lgb_model = Pipeline([
        ('sampler', RandomUnderSampler(sampling_strategy=sampling_ratio, random_state=seed)),
        ('classifier', lgb.LGBMClassifier(**lgb_params)),
    ])

##### 10.1.2 CatBoost

In [ ]:
if CFG.use_tabler and ((not CFG.is_infer) or (not CFG.image_pred_as_tabler_feature)):
    cb_params = {
        'loss_function':     'Logloss',
        'iterations':        200,
        'verbose':           False,
        'random_state':      seed,
        'max_depth':         7, 
        'learning_rate':     0.06936242010150652, 
        'scale_pos_weight':  2.6149345838209532, 
        'l2_leaf_reg':       6.216113851699493, 
        'subsample':         0.6249261779711819, 
        'min_data_in_leaf':  24,
        'cat_features':      cat_cols,
    }

    cb_model = Pipeline([
        ('sampler', RandomUnderSampler(sampling_strategy=sampling_ratio, random_state=seed)),
        ('classifier', cb.CatBoostClassifier(**cb_params)),
    ])

##### 10.1.3 XGBoost

In [ ]:
if CFG.use_tabler and ((not CFG.is_infer) or (not CFG.image_pred_as_tabler_feature)):
    xgb_params = {
        'enable_categorical': True,
        'tree_method':        'hist',
        'random_state':       seed,
        'learning_rate':      0.08501257473292347, 
        'lambda':             8.879624125465703, 
        'alpha':              0.6779926606782505, 
        'max_depth':          6, 
        'subsample':          0.6012681388711075, 
        'colsample_bytree':   0.8437772277074493, 
        'colsample_bylevel':  0.5476090898823716, 
        'colsample_bynode':   0.9928601203635129, 
        'scale_pos_weight':   3.29440313334688,
    }

    xgb_model = Pipeline([
        ('sampler', RandomUnderSampler(sampling_strategy=sampling_ratio, random_state=seed)),
        ('classifier', xgb.XGBClassifier(**xgb_params)),
    ])

In [ ]:
if CFG.use_tabler and ((not CFG.is_infer) or (not CFG.image_pred_as_tabler_feature)):
    estimator = VotingClassifier([
        ('lgb', lgb_model), ('cb', cb_model), ('xgb', xgb_model),
    ], voting='soft')

### 10.2 CV

In [ ]:
if CFG.use_tabler and ((not CFG.is_infer)):
    # Assuming df_train already has a 'fold' column with values 0-4
    X = train_meta[feature_cols]
    y = train_meta[target_col]

    # Create a PredefinedSplit object using the 'fold' column
    fold_indices = train_meta['fold'].values
    cv = PredefinedSplit(fold_indices)

    # if CFG.is_infer: # inference
    #     with open(str(CFG.OUTPUT_DIR / f'voting_classifier.pkl'), 'rb') as file:
    #         estimator = pickle.load(file)
    # Perform cross-validation
    val_score = cross_val_score(
        estimator=estimator, 
        X=X, y=y, 
        cv=cv,
        scoring=custom_metric,
    )

    print(np.mean(val_score), val_score)
    if CFG.enable_wandb:
        wandb.log({"tabler_CV": np.mean(val_score)})

### 10.3 Training

In [ ]:
if CFG.use_tabler and ((not CFG.is_infer) or (not CFG.image_pred_as_tabler_feature)):
    X, y = train_meta[feature_cols], train_meta[target_col]

    estimator.fit(X, y)
    # save
    if not CFG.is_kaggle:
        with open(str(CFG.OUTPUT_DIR / f'voting_classifier.pkl'), 'wb') as file:
            pickle.dump(estimator, file)


### 10.4 Prediction

In [ ]:
if CFG.use_tabler:
    
    if CFG.is_infer: # inference
        with open(str(CFG.OUTPUT_DIR / f'voting_classifier.pkl'), 'rb') as file:
            estimator = pickle.load(file)
        df_subm['target'] = estimator.predict_proba(test_meta[feature_cols])[:, 1]
        df_subm.to_csv('submission.csv')
    else: # training
        df_subm['target'] = estimator.predict_proba(test_meta[feature_cols])[:, 1]
        
    if CFG.ensemble_image_table:
        # image prediction
        print('image pred')
        display(pred_df)
        # table prediction
        print('table pred')
        display(df_subm)
        # blending
        df_subm['target'] = (df_subm['target'].values * 0.5) + (pred_df['target'].values * 0.5)
        df_subm.to_csv('submission.csv')
        
        
        print('blended pred')
        display(df_subm.head())

### Score

In [ ]:
if CFG.ensemble_image_table:

    # make true array
    label_arr = train_meta['target']
    # one-hot
    # ture_arr = np.zeros((label_arr.size, CFG.n_classes))
    # ture_arr[np.arange(label_arr.size), label_arr] = 1
    ture_arr = pd.DataFrame(label_arr)

    # pred
    pred = estimator.predict_proba(train_meta[feature_cols])[:, 1]
    pred = pd.DataFrame(pred)

    micro_roc_auc_ovr = comp_score(
        ture_arr,
        pred,
        ""
    )

    print(f"CV: Micro-averaged One-vs-Rest ROC AUC score:\n{micro_roc_auc_ovr:.10f}")
    if CFG.enable_wandb:
        wandb.log({"CV": micro_roc_auc_ovr})

### Importance Visualization

In [ ]:
for model in estimator.estimators_:
    model = model['classifier']
    model_name = str(model)
    model_name_list = ['LGBM', 'catboost', 'XGB']
    number_of_show = 10
    if model_name_list[0] in model_name:
        lgb.plot_importance(model, max_num_features=number_of_show, importance_type='split')
        plt.title('LGBM Feature Importance - Number of Splits')
        plt.show()
        lgb.plot_importance(model, max_num_features=number_of_show, importance_type='gain')
        plt.title('LGBM Feature Importance - Gain')
        plt.show()
    if model_name_list[1] in model_name:
        # PVC
        pve_importances = model.get_feature_importance(type='PredictionValuesChange') # default type
        sorted_indices = np.argsort(pve_importances)[-number_of_show:]
        plt.figure(figsize=(10, 6))
        plt.barh(np.array(feature_cols)[sorted_indices], pve_importances[sorted_indices])
        plt.xlabel('Importance')
        plt.ylabel('Feature')
        plt.title('CatBoost Sorted Prediction Values Change (PVE) Feature Importance')
        plt.show()
        # # Shapley
        # shap_values = model.get_feature_importance(cb.Pool(X,y), type='ShapValues')
        # shap_importances = np.mean(np.abs(shap_values[:, :-1]), axis=0)
        # sorted_indices = np.argsort(shap_importances)[-number_of_show:]
        # plt.figure(figsize=(10, 6))
        # plt.barh(np.array(feature_cols)[sorted_indices], shap_importances[sorted_indices])
        # plt.xlabel('Importance')
        # plt.ylabel('Feature')
        # plt.title('CatBoost Sorted Shapley Values Feature Importance')
        # plt.show()
    if model_name_list[2] in model_name:
        xgb.plot_importance(model, importance_type='weight', title='XGB Feature Importance by Weight', xlabel='Importance', ylabel='Feature', max_num_features=number_of_show)
        plt.show()

        xgb.plot_importance(model, importance_type='gain', title='XGB Feature Importance by Gain', xlabel='Importance', ylabel='Feature', max_num_features=number_of_show)
        plt.show()

        xgb.plot_importance(model, importance_type='cover', title='XGB Feature Importance by Cover', xlabel='Importance', ylabel='Feature', max_num_features=number_of_show)
        plt.show()



### save dfs

In [ ]:
# save the dfs
if (not CFG.is_infer) and (not CFG.is_kaggle):
    train_meta.to_parquet(CFG.OUTPUT_DIR / 'train_meta.parquet', index=False)
    test_meta.to_parquet(CFG.OUTPUT_DIR / 'test_meta.parquet', index=False)